In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
        SparkSession.builder
        .appName("Test_task_2")
        .getOrCreate()
        )

file_path_books = ("/home/ilya/github/"
                   "DE_course_repo/python_projects/"
                   "PySpark/data/books.csv")

file_path_authors = ("/home/ilya/github/"
                     "DE_course_repo/python_projects/"
                     "PySpark/data/authors.csv")            

books_df = spark.read.csv(file_path_books,
                            header=True,
                            inferSchema=True)

authors_df = spark.read.csv(file_path_authors,
                            header=True,
                            inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 16:55:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/16 16:55:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
books_df.show(5)
authors_df.show(5)

books_df.printSchema()
authors_df.printSchema()

+-------+------+---------+-----------+-----+------------+
|book_id| title|author_id|      genre|price|publish_date|
+-------+------+---------+-----------+-----+------------+
|      1|Book_1|        2|    Mystery|73.57|  1980-12-31|
|      2|Book_2|        1|Non-Fiction| 41.1|  1982-12-31|
|      3|Book_3|       10|    Fiction|10.63|  1984-12-31|
|      4|Book_4|        9|Non-Fiction|46.31|  1986-12-31|
|      5|Book_5|        7|    Science|31.13|  1988-12-31|
+-------+------+---------+-----------+-----+------------+
only showing top 5 rows

+---------+--------+----------+-------+
|author_id|    name|birth_date|country|
+---------+--------+----------+-------+
|        1|Author_1|1960-12-31|  India|
|        2|Author_2|1965-12-31| Canada|
|        3|Author_3|1970-12-31|    USA|
|        4|Author_4|1975-12-31|     UK|
|        5|Author_5|1980-12-31|    USA|
+---------+--------+----------+-------+
only showing top 5 rows

root
 |-- book_id: integer (nullable = true)
 |-- title: string (nul

In [3]:
merged_df = books_df.join(authors_df, on="author_id", how='left')
merged_df.show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|author_id|book_id|  title|      genre|price|publish_date|     name|birth_date|  country|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|        2|      1| Book_1|    Mystery|73.57|  1980-12-31| Author_2|1965-12-31|   Canada|
|        1|      2| Book_2|Non-Fiction| 41.1|  1982-12-31| Author_1|1960-12-31|    India|
|       10|      3| Book_3|    Fiction|10.63|  1984-12-31|Author_10|2005-12-31|    India|
|        9|      4| Book_4|Non-Fiction|46.31|  1986-12-31| Author_9|2000-12-31|Australia|
|        7|      5| Book_5|    Science|31.13|  1988-12-31| Author_7|1990-12-31|      USA|
|        4|      6| Book_6|Non-Fiction| 83.7|  1990-12-31| Author_4|1975-12-31|       UK|
|        6|      7| Book_7|Non-Fiction|40.36|  1992-12-31| Author_6|1985-12-31|      USA|
|        2|      8| Book_8|Non-Fiction|84.48|  1994-12-31| Author_2|1965-12-31|   Canada|
|        7

In [6]:
merged_df.printSchema()

root
 |-- author_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country: string (nullable = true)



In [11]:
(merged_df.groupBy("author_id", "name").agg(F.sum("price")
.alias("total_price")).orderBy(F.desc("total_price")).limit(5).show())

+---------+--------+-----------+
|author_id|    name|total_price|
+---------+--------+-----------+
|        2|Author_2|     231.97|
|        7|Author_7|     132.66|
|        1|Author_1|     111.86|
|        8|Author_8|     107.16|
|        5|Author_5|      88.83|
+---------+--------+-----------+



In [14]:
merged_df.groupby("genre").agg(F.count("title").alias("count_of_books")).orderBy(F.desc("count_of_books")).show()

+-----------+--------------+
|      genre|count_of_books|
+-----------+--------------+
|Non-Fiction|             9|
|    Science|             3|
|    Fiction|             3|
|    Fantasy|             3|
|    Mystery|             2|
+-----------+--------------+



In [15]:
merged_df.groupBy("author_id", "name").agg(F.avg("price").alias("avg_price")).orderBy(F.desc("avg_price")).show()

+---------+---------+-----------------+
|author_id|     name|        avg_price|
+---------+---------+-----------------+
|        5| Author_5|            88.83|
|        4| Author_4|             83.7|
|        2| Author_2|          57.9925|
|        9| Author_9|            46.31|
|        7| Author_7|            44.22|
|        6| Author_6|           43.965|
|        1| Author_1|37.28666666666667|
|        8| Author_8|            35.72|
|       10|Author_10|           21.165|
+---------+---------+-----------------+



In [16]:
merged_df = merged_df.withColumn("year", F.year("publish_date"))
merged_df.show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+----+
|author_id|book_id|  title|      genre|price|publish_date|     name|birth_date|  country|year|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+----+
|        2|      1| Book_1|    Mystery|73.57|  1980-12-31| Author_2|1965-12-31|   Canada|1980|
|        1|      2| Book_2|Non-Fiction| 41.1|  1982-12-31| Author_1|1960-12-31|    India|1982|
|       10|      3| Book_3|    Fiction|10.63|  1984-12-31|Author_10|2005-12-31|    India|1984|
|        9|      4| Book_4|Non-Fiction|46.31|  1986-12-31| Author_9|2000-12-31|Australia|1986|
|        7|      5| Book_5|    Science|31.13|  1988-12-31| Author_7|1990-12-31|      USA|1988|
|        4|      6| Book_6|Non-Fiction| 83.7|  1990-12-31| Author_4|1975-12-31|       UK|1990|
|        6|      7| Book_7|Non-Fiction|40.36|  1992-12-31| Author_6|1985-12-31|      USA|1992|
|        2|      8| Book_8|Non-Fiction|84.48|  199

In [18]:
merged_df.filter(merged_df["year"] >= 2000).orderBy(F.desc("price")).show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+----+
|author_id|book_id|  title|      genre|price|publish_date|     name|birth_date|  country|year|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+----+
|        7|     20|Book_20|    Mystery|91.48|  2018-12-31| Author_7|1990-12-31|      USA|2018|
|        5|     19|Book_19|    Science|88.83|  2016-12-31| Author_5|1980-12-31|      USA|2016|
|        8|     15|Book_15|    Fantasy| 60.0|  2008-12-31| Author_8|1995-12-31|Australia|2008|
|        6|     17|Book_17|    Fantasy|47.57|  2012-12-31| Author_6|1985-12-31|      USA|2012|
|        1|     18|Book_18|Non-Fiction|43.92|  2014-12-31| Author_1|1960-12-31|    India|2014|
|        2|     16|Book_16|    Fiction|36.22|  2010-12-31| Author_2|1965-12-31|   Canada|2010|
|       10|     11|Book_11|Non-Fiction| 31.7|  2000-12-31|Author_10|2005-12-31|    India|2000|
|        8|     12|Book_12|Non-Fiction|31.02|  200

In [19]:
spark.stop()